In [86]:
import torch as tc 
from torch import optim, nn  
from torch.utils.data import TensorDataset, DataLoader, random_split 

from torchvision import transforms as T 
from torchvision import datasets 

from torchmetrics import Accuracy 

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

from tqdm import tqdm

In [87]:
device = 'cuda' if tc.cuda.is_available() else 'cpu'

In [88]:
transform = T.Compose([T.ToTensor(), T.Resize(size = (512, 512))])

In [89]:
train_dataset = datasets.ImageFolder('/home/ahmadrezabaqerzadeh/CovidDataSet/train/', transform = transform)
valid_dataset = datasets.ImageFolder('/home/ahmadrezabaqerzadeh/CovidDataSet/validation/', transform = transform)

In [90]:
train_dataset

Dataset ImageFolder
    Number of datapoints: 73
    Root location: /home/ahmadrezabaqerzadeh/CovidDataSet/train/
    StandardTransform
Transform: Compose(
               ToTensor()
               Resize(size=(512, 512), interpolation=bilinear, max_size=None, antialias=None)
           )

In [91]:
valid_dataset

Dataset ImageFolder
    Number of datapoints: 21
    Root location: /home/ahmadrezabaqerzadeh/CovidDataSet/validation/
    StandardTransform
Transform: Compose(
               ToTensor()
               Resize(size=(512, 512), interpolation=bilinear, max_size=None, antialias=None)
           )

In [92]:
train_loader = DataLoader(train_dataset, batch_size = 20, shuffle = True)
valid_loader = DataLoader(valid_dataset, batch_size = 21, shuffle = False)

In [93]:
x, y = next(iter(train_loader))

In [94]:
class CNNModel(nn.Module):
    
    def __init__(self, num_filters, kernel_size, num_classes):
        super(CNNModel, self).__init__()
        
        self.conv1 = nn.LazyConv2d(out_channels = num_filters[0], kernel_size = kernel_size[0], stride = 1, padding = 1, bias = True)
        self.bn1   = nn.LazyBatchNorm2d()
        self.mpool1 = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.LazyConv2d(out_channels = num_filters[1], kernel_size = kernel_size[1], stride = 1, padding = 1, bias = True)
        self.bn2   = nn.LazyBatchNorm2d()
        self.mpool2 = nn.MaxPool2d(2, 2)
        
        self.conv3 = nn.LazyConv2d(out_channels = num_filters[2], kernel_size = kernel_size[2], stride = 1, padding = 1, bias = True)
        self.bn3   = nn.LazyBatchNorm2d()
        self.mpool3 = nn.MaxPool2d(2, 2)
        
        self.conv4 = nn.LazyConv2d(out_channels = num_filters[3], kernel_size = kernel_size[3], stride = 1, padding = 1, bias = True)
        self.bn4   = nn.LazyBatchNorm2d()
        self.mpool4 = nn.MaxPool2d(2, 2)
        
        self.conv5 = nn.LazyConv2d(out_channels = num_filters[4], kernel_size = kernel_size[4], stride = 1, padding = 1, bias = True)
        self.bn5   = nn.LazyBatchNorm2d()
        self.apool = nn.AdaptiveAvgPool2d((1, 1))
        
        self.fc = nn.LazyLinear(num_classes)
        
    def forward(self, x):
        
        y = self.conv1(x)
        y = self.bn1(y)
        y = self.mpool1(y)
        y = y.relu()
        y = nn.Dropout(0.3)(y)
        
        y = self.conv2(y)
        y = self.bn2(y)
        y = self.mpool2(y)
        y = y.relu()
        y = nn.Dropout(0.3)(y)
        
        y = self.conv3(y)
        y = self.bn3(y)
        y = self.mpool3(y)
        y = y.relu()
        y = nn.Dropout(0.3)(y)
        
        y = self.conv4(y)
        y = self.bn4(y)
        y = self.mpool4(y)
        y = y.relu()
        y = nn.Dropout(0.3)(y)
        
        y = self.conv5(y)
        y = self.bn5(y)
        y = self.apool(y)
        y = y.relu()
        
        y = y.mean(dim = 1)
        
        y = y.flatten(1)
        
        y = self.fc(y)
        y = nn.Sigmoid()(y)
        
        return y

In [95]:
class TransformerModel(nn.Module):
    
    def __init__(self,d_model, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward, dropout, activation, num_classes):
        super(TransformerModel, self).__init__()
        
        self.encoder = nn.Transformer(d_model, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward, dropout, activation, batch_first = True).encoder 
        
        self.fc = nn.LazyLinear(num_classes)
        
        
        
    def forward(self, x):
        
        x = x.mean(dim = 1)
        
        y = self.encoder(x)
        
        y = y.mean(dim = 1)
        
        y = self.fc(y)
        y = nn.Sigmoid()(y)
        
        return y

In [131]:
class FinallyModel(nn.Module):
    
    def __init__(self):
        super(FinallyModel, self).__init__()
        
        self.fc = nn.LazyLinear(1)
        
    def forward(self, x):
       
        y1 = best_model1[0](x)
        
        y2 = best_model2[0](x)
        
        y3 = best_model3[0](x)
        
        z = tc.cat([y1, y2, y3], dim = 1)
        
        y = self.fc(z)
        y = nn.Sigmoid()(y)
        
        return y

In [97]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [98]:
def train_one_epoch(model, train_loader, loss_func, optimizer, epoch):
    
    train_loss = AverageMeter()
    train_acc = Accuracy(task = 'binary').to(device)
    
    with tqdm(train_loader, unit = 'batch') as tepoch:
        
        if epoch is not None:
            tepoch.set_description(f'Epoch={epoch}')
        
        for x, y in tepoch:
            
            x = x.to(device)
            y = y.to(device)
            
            yp = model(x)
            loss = loss_func(yp.squeeze(), y.type(tc.float))
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
            train_loss.update(loss.item())
            train_acc(yp.squeeze(), y)
            
            tepoch.set_postfix(loss = train_loss.avg, accuracy = train_acc.compute().item() * 100)
            
    return model, train_loss.avg, train_acc.compute().item()

In [99]:
def validation(model, valid_loaedr, loss_func):
    model.eval()

    valid_loss=AverageMeter()
    valid_acc=Accuracy(task='binary').to(device)

    for x, y in valid_loaedr:

        x=x.to(device)
        y=y.to(device)

        yp=model(x)
        loss=loss_func(yp.squeeze(), y.type(tc.float))

        valid_loss.update(loss.item())
        valid_acc(yp.squeeze(), y)

    print(f'valid loss={valid_loss.avg:.4}, accuracy={valid_acc.compute() * 100 :.4}')
    print()

    return valid_loss.avg, valid_acc.compute().item()

In [108]:
model = CNNModel([4, 8, 16, 32, 128], [3, 3, 3, 3, 3],1).to(device)

In [109]:
optimizer = optim.Adam(model.parameters(), lr = 0.0015, betas = [0.9, 0.99])
loss_func = nn.BCELoss()

In [110]:
best_loss = tc.inf 
best_model1 = [0]

In [111]:
N = 10

for i in range(N):

  model, train_loss, train_acc = train_one_epoch(model, train_loader, loss_func, optimizer, i)
  valid_loss, valid_acc = validation(model,valid_loader,loss_func)

  if valid_loss<best_loss:
    best_loss = valid_loss
    best_model1[0] = model
    print('model saved!')

Epoch=0: 100%|██████████| 4/4 [00:03<00:00,  1.06batch/s, accuracy=45.2, loss=0.69] 


valid loss=0.6948, accuracy=38.1

model saved!


Epoch=1: 100%|██████████| 4/4 [00:03<00:00,  1.02batch/s, accuracy=57.5, loss=0.692]


valid loss=0.6905, accuracy=61.9

model saved!


Epoch=2: 100%|██████████| 4/4 [00:03<00:00,  1.11batch/s, accuracy=76.7, loss=0.682]


...

valid loss=0.6888, accuracy=61.9



Epoch=8: 100%|██████████| 4/4 [00:03<00:00,  1.05batch/s, accuracy=76.7, loss=0.542]


valid loss=0.7657, accuracy=61.9



Epoch=9: 100%|██████████| 4/4 [00:03<00:00,  1.02batch/s, accuracy=76.7, loss=0.549]


valid loss=0.7717, accuracy=61.9



In [ ]:
model = CNNModel([8, 16, 32, 64, 128], [3, 3, 3, 3, 3],1).to(device)

In [ ]:
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.9)
loss_func = nn.BCELoss()

In [ ]:
best_loss = tc.inf 
best_model2 = [0]

In [ ]:
N = 30

for i in range(N):

  model, train_loss, train_acc = train_one_epoch(model, train_loader, loss_func, optimizer, i)
  valid_loss, valid_acc = validation(model,valid_loader,loss_func)

  if valid_loss<best_loss:
    best_loss = valid_loss
    best_model2[0] = model
    print('model saved!')

Epoch=0: 100%|██████████| 4/4 [00:02<00:00,  1.51batch/s, accuracy=76.7, loss=0.675]


valid loss=0.6851, accuracy=61.9

model saved!


Epoch=1: 100%|██████████| 4/4 [00:02<00:00,  1.56batch/s, accuracy=76.7, loss=0.669]


valid loss=0.6803, accuracy=61.9

model saved!


Epoch=2: 100%|██████████| 4/4 [00:02<00:00,  1.47batch/s, accuracy=76.7, loss=0.656]


...

valid loss=0.7107, accuracy=61.9



Epoch=28: 100%|██████████| 4/4 [00:02<00:00,  1.78batch/s, accuracy=76.7, loss=0.551]


valid loss=0.7118, accuracy=61.9



Epoch=29: 100%|██████████| 4/4 [00:02<00:00,  1.77batch/s, accuracy=76.7, loss=0.551]


valid loss=0.7126, accuracy=61.9



In [ ]:
model = TransformerModel(512, 4, 4, 0, 2048, 0.3, nn.ReLU(), 1).to(device)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr = 0.0001)
loss_func = nn.BCELoss()

In [ ]:
best_loss = tc.inf 
best_model3 = [0]

In [ ]:
N = 30

for i in range(N):

  model, train_loss, train_acc = train_one_epoch(model, train_loader, loss_func, optimizer, i)
  valid_loss, valid_acc = validation(model,valid_loader,loss_func)

  if valid_loss<best_loss:
    best_loss = valid_loss
    best_model3[0] = model
    print('model saved!')

Epoch=0: 100%|██████████| 4/4 [00:04<00:00,  1.11s/batch, accuracy=76.7, loss=0.629]


valid loss=0.4525, accuracy=80.95

model saved!


Epoch=1: 100%|██████████| 4/4 [00:04<00:00,  1.10s/batch, accuracy=75.3, loss=0.528]


valid loss=0.4856, accuracy=61.9



Epoch=2: 100%|██████████| 4/4 [00:04<00:00,  1.08s/batch, accuracy=84.9, loss=0.307]


...

valid loss=0.2311, accuracy=95.24



Epoch=28: 100%|██████████| 4/4 [00:04<00:00,  1.12s/batch, accuracy=100, loss=8.72e-5] 


valid loss=0.2257, accuracy=95.24



Epoch=29: 100%|██████████| 4/4 [00:04<00:00,  1.10s/batch, accuracy=100, loss=7.91e-5]


valid loss=0.2206, accuracy=95.24



In [132]:
device = 'cpu'

In [133]:
best_model1[0] = best_model1[0].to(device)
best_model2[0] = best_model2[0].to(device)
best_model3[0] = best_model3[0].to(device)

In [154]:
model = FinallyModel().to(device)

In [155]:
optimizer = optim.SGD(model.parameters(), lr = 0.05, momentum = 0.9)
loss_func = nn.BCELoss()

In [156]:
best_loss = tc.inf 
best_model4 = [0]

In [157]:
N = 30
for i in range(N):

  model, train_loss, train_acc = train_one_epoch(model, train_loader, loss_func, optimizer, i)
  valid_loss, valid_acc = validation(model,valid_loader,loss_func)

  if valid_loss<best_loss:
    best_loss = valid_loss
    best_model4[0] = model
    print('model saved!')

Epoch=0: 100%|██████████| 4/4 [00:22<00:00,  5.64s/batch, accuracy=76.7, loss=0.728]


valid loss=0.7701, accuracy=61.9

model saved!


Epoch=1: 100%|██████████| 4/4 [00:18<00:00,  4.67s/batch, accuracy=76.7, loss=0.681]


valid loss=0.7333, accuracy=61.9

model saved!


Epoch=2: 100%|██████████| 4/4 [00:19<00:00,  4.83s/batch, accuracy=76.7, loss=0.615]


...

valid loss=0.1993, accuracy=95.24

model saved!


Epoch=28: 100%|██████████| 4/4 [00:20<00:00,  5.08s/batch, accuracy=100, loss=0.0915]


valid loss=0.1971, accuracy=95.24

model saved!


Epoch=29: 100%|██████████| 4/4 [00:20<00:00,  5.04s/batch, accuracy=100, loss=0.0895]


valid loss=0.1949, accuracy=95.24

model saved!


In [158]:
N = 15
for i in range(N):

  model, train_loss, train_acc = train_one_epoch(model, train_loader, loss_func, optimizer, i)
  valid_loss, valid_acc = validation(model,valid_loader,loss_func)

  if valid_loss<best_loss:
    best_loss = valid_loss
    best_model4[0] = model
    print('model saved!')

Epoch=0: 100%|██████████| 4/4 [00:20<00:00,  5.13s/batch, accuracy=100, loss=0.0857]


valid loss=0.1929, accuracy=95.24

model saved!


Epoch=1: 100%|██████████| 4/4 [00:20<00:00,  5.02s/batch, accuracy=100, loss=0.0829]


valid loss=0.1913, accuracy=95.24

model saved!


Epoch=2: 100%|██████████| 4/4 [00:20<00:00,  5.24s/batch, accuracy=100, loss=0.0813]


...

Epoch=13: 100%|██████████| 4/4 [00:32<00:00,  8.07s/batch, accuracy=100, loss=0.0592]


valid loss=0.179, accuracy=95.24

model saved!


Epoch=14: 100%|██████████| 4/4 [00:34<00:00,  8.52s/batch, accuracy=100, loss=0.0578]


valid loss=0.1785, accuracy=95.24

model saved!


Epoch=15:  25%|██▌       | 1/4 [00:08<00:26,  8.72s/batch, accuracy=100, loss=0.0646]